In [1]:
import pandas as pd
import altair as alt
import chess.pgn
import urllib.request

In [2]:
game_id = 'd8CgIg1W'
url = 'https://lichess.org/game/export/' + game_id
pgn_filename = "game.pgn"

urllib.request.urlretrieve(url, pgn_filename)

('game.pgn', <http.client.HTTPMessage at 0x7f663363b7d0>)

In [3]:
with open(pgn_filename) as pgn_f:
  game = chess.pgn.read_game(pgn_f)

In [4]:
print(game)

[Event "Rated Rapid game"]
[Site "https://lichess.org/d8CgIg1W"]
[Date "2021.04.19"]
[Round "?"]
[White "DOYGARS"]
[Black "Hryts"]
[Result "0-1"]
[BlackElo "1921"]
[BlackRatingDiff "+9"]
[ECO "D00"]
[Opening "Queen's Pawn Game: Mason Variation"]
[Termination "Time forfeit"]
[TimeControl "600+0"]
[UTCDate "2021.04.19"]
[UTCTime "14:22:09"]
[Variant "Standard"]
[WhiteElo "2061"]
[WhiteRatingDiff "-8"]

1. d4 { [%eval 0.0] [%clk 0:10:00] } 1... d5 { [%eval 0.37] [%clk 0:10:00] } 2. Bf4 { [%eval 0.0] [%clk 0:09:58] } 2... f5 { [%eval 1.05] [%clk 0:09:56] } 3. Nf3 { [%eval 0.92] [%clk 0:09:53] } 3... h6 { [%eval 2.43] [%clk 0:09:55] } 4. h3 { [%eval 1.4] [%clk 0:09:42] } 4... g5 { [%eval 1.47] [%clk 0:09:54] } 5. Be5 { [%eval 1.25] [%clk 0:09:20] } 5... Rh7 { [%eval 2.14] [%clk 0:09:50] } 6. e3 { [%eval 2.51] [%clk 0:09:05] } 6... Nc6 { [%eval 2.46] [%clk 0:09:49] } 7. Bh2 { [%eval 1.21] [%clk 0:08:50] } 7... Bg7 { [%eval 3.09] [%clk 0:09:46] } 8. c3 { [%eval 2.4] [%clk 0:08:47] } 8... b6 {

In [5]:
PIECES_URL = {
    'b': 'https://raw.githubusercontent.com/Hryts/visualization_project/92e2a29a8a83230b64266dfd57345177f0350290/images/b_bishop.svg',
    'k': 'https://raw.githubusercontent.com/Hryts/visualization_project/92e2a29a8a83230b64266dfd57345177f0350290/images/b_king.svg',
    'n': 'https://raw.githubusercontent.com/Hryts/visualization_project/92e2a29a8a83230b64266dfd57345177f0350290/images/b_knight.svg',
    'p': 'https://raw.githubusercontent.com/Hryts/visualization_project/92e2a29a8a83230b64266dfd57345177f0350290/images/b_pawn.svg',
    'q': 'https://raw.githubusercontent.com/Hryts/visualization_project/92e2a29a8a83230b64266dfd57345177f0350290/images/b_queen.svg',
    'r': 'https://raw.githubusercontent.com/Hryts/visualization_project/92e2a29a8a83230b64266dfd57345177f0350290/images/b_rook.svg',

    'B': 'https://raw.githubusercontent.com/Hryts/visualization_project/92e2a29a8a83230b64266dfd57345177f0350290/images/w_bishop.svg',
    'K': 'https://raw.githubusercontent.com/Hryts/visualization_project/92e2a29a8a83230b64266dfd57345177f0350290/images/w_king.svg',
    'N': 'https://raw.githubusercontent.com/Hryts/visualization_project/92e2a29a8a83230b64266dfd57345177f0350290/images/w_knight.svg',
    'P': 'https://raw.githubusercontent.com/Hryts/visualization_project/92e2a29a8a83230b64266dfd57345177f0350290/images/w_pawn.svg',
    'Q': 'https://raw.githubusercontent.com/Hryts/visualization_project/92e2a29a8a83230b64266dfd57345177f0350290/images/w_queen.svg',
    'R': 'https://raw.githubusercontent.com/Hryts/visualization_project/92e2a29a8a83230b64266dfd57345177f0350290/images/w_rook.svg'
}

BOARD_URL = 'https://raw.githubusercontent.com/Hryts/visualization_project/efd673ac67880893d606d7fffde420506e858b73/images/svgs/board.svg'

PREV_MOVE_URL = 'https://raw.githubusercontent.com/Hryts/visualization_project/2cac54259d6c1aab6a034256b4c17e4ea2d5a3fa/images/green_frame.svg'

In [6]:
board_size = {'width': 500, 'height': 500}
piece_size = {'width':  board_size['width']  / 8,
              'height': board_size['height'] / 8}

In [7]:
board_source = pd.DataFrame.from_records([
      {"x": 0.0, "y": 0.0, 
       'w': board_size['width'], 'h':board_size['height'],  
       "img": BOARD_URL},
])

board_chart = alt.Chart(board_source).mark_image(
    width=board_size['width'],
    height=board_size['height']
).encode(
    x=alt.X('x', axis=None),
    y=alt.Y('y', axis=None),
    x2='w',
    y2='h',
    url='img'
)

In [8]:
def rearange_fen(fen_str):
    cells = fen_str.split('\n')
    cells.reverse()
    cells = [item for sublist in cells for item in sublist.split()]
    return cells


def fen_to_dict(fen_str, p_size=piece_size):
    res = {}

    cells = rearange_fen(fen_str)

    for i, cell in enumerate(cells):
       x_local = i % 8
       y_local = i // 8

       x_abs = (x_local + 0.5) * p_size['width']
       y_abs = (y_local + 0.5) * p_size['height']
       
       if (cell != '.'): 
         res[(x_abs, y_abs)] = cell
    return res


def chess_notation_to_coords(current_move, p_size=piece_size):
    if not current_move:
        return
    xs = 'abcdefgh'
    current_move = str(current_move)[:2]
    x = (xs.index(current_move[0]) + 0.5) * p_size['width']
    y = (int(current_move[1]) - 1 + 0.5) * p_size['height']
    return (x, y)


def prev_move_data(prev_move):
    prev_move_coords = chess_notation_to_coords(prev_move)
    if prev_move_coords:
      prev_move_x = prev_move_coords[0]
      prev_move_y = prev_move_coords[1]
      prev_move_url = PREV_MOVE_URL
    else:
      prev_move_x = board_size['width'] / 2
      prev_move_y = board_size['height'] / 2
      prev_move_url = ''
    return prev_move_x, prev_move_y, prev_move_url

In [9]:
py_chess_board = game.board()

states = pd.DataFrame()

counter = 0
prev_move = None

for move in game.main_line():
    fen = str(py_chess_board)
    state = fen_to_dict(fen)

    prev_move_x, prev_move_y, prev_move_url = prev_move_data(prev_move)

    for coords in state:
        row = {'move_n': counter, 
               'x': coords[0], 
               'y': coords[1], 
               'piece_url': PIECES_URL[state[coords]],
               'prev_move_x': prev_move_x,
               'prev_move_y': prev_move_y,
               'prev_move_url': prev_move_url}
        states = states.append(row, ignore_index=True)

    counter += 1
    prev_move = move
    py_chess_board.push(move)

In [10]:
states

,move_n,piece_url,prev_move_url,prev_move_x,prev_move_y,x,y
0,0.0,https://raw.githubusercontent.com/Hryts/visual...,,250.00,250.00,31.25,31.25
1,0.0,https://raw.githubusercontent.com/Hryts/visual...,,250.00,250.00,93.75,31.25
2,0.0,https://raw.githubusercontent.com/Hryts/visual...,,250.00,250.00,156.25,31.25
3,0.0,https://raw.githubusercontent.com/Hryts/visual...,,250.00,250.00,218.75,31.25
4,0.0,https://raw.githubusercontent.com/Hryts/visual...,,250.00,250.00,281.25,31.25
...,...,...,...,...,...,...,...
2046,105.0,https://raw.githubusercontent.com/Hryts/visual...,https://raw.githubusercontent.com/Hryts/visual...,281.25,156.25,218.75,93.75
2047,105.0,https://raw.githubusercontent.com/Hryts/visual...,https://raw.githubusercontent.com/Hryts/visual...,281.25,156.25,93.75,156.25
2048,105.0,https://raw.githubusercontent.com/Hryts/visual...,https://raw.githubusercontent.com/Hryts/visual...,281.25,156.25,156.25,343.75
2049,105.0,https://raw.githubusercontent.com/Hryts/visual...,https://raw.githubusercontent.com/Hryts/visual...,281.25,156.25,156.25,406.25


In [11]:
slider = alt.binding_range(min=0, max=counter-1, step=1, name='select move: ')
selector_date = alt.selection_single(name='move_n', fields=['move_n'], bind=slider, init={'move_n': 0})


pieces_chart = alt.Chart(states).transform_filter(selector_date).mark_image(
    height=piece_size['width'],
    width=piece_size['height']
).encode(
    x=alt.X('x:Q'),
    y=alt.Y('y:Q'),
    url='piece_url:N'
)

prev_move_chart = alt.Chart(states).transform_filter(selector_date).mark_image(
    height=piece_size['width'],
    width=piece_size['height']
).encode(
    x=alt.X('prev_move_x:Q'),
    y=alt.Y('prev_move_y:Q'),
    url='prev_move_url:N'
)

(board_chart + pieces_chart + prev_move_chart).add_selection(selector_date).properties(
    width=board_size['width'],
    height=board_size['height'],
    title=alt.TitleParams(
        text=f'white - {game.headers["White"]} ({game.headers["WhiteElo"]}) vs {game.headers["Black"]} ({game.headers["BlackElo"]}) - black',
        subtitle=[game.headers["Event"]],
        fontSize=20,
        subtitleFontSize=16,
        dy=-25,
        dx=0
    )
)

alt.LayerChart(...)